In [1]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

In [2]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}
            
    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping
            

        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)
             
        return X_encoded 

In [3]:
syn_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043_set5.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,-0.341812,0.170435,0.170435,0.593641,0.170435,0.000000,-0.341812,-0.341812,-1.088625,0.000000,0.170435,-0.341812,0.593641,-0.341812,0.593641,0.000000,0.593641,0.170435,-0.341812,-1.088625
1,-0.341812,0.981444,1.365898,1.365898,0.170435,2.261507,1.365898,0.981444,1.777345,1.777345,0.170435,0.981444,0.593641,0.981444,0.593641,1.365898,0.593641,0.981444,2.261507,0.593641
2,-0.341812,0.170435,-1.088625,0.000000,-1.088625,-0.341812,0.593641,1.365898,-0.341812,0.170435,0.170435,0.000000,-0.341812,-1.088625,0.170435,-0.341812,-0.341812,0.170435,0.981444,-0.341812
3,-0.341812,0.981444,0.000000,0.170435,-0.341812,-0.341812,-0.341812,-0.341812,-1.088625,0.170435,0.000000,-0.341812,-0.341812,0.593641,-1.088625,0.000000,-0.341812,-0.341812,0.000000,0.000000
4,-0.341812,-1.088625,0.170435,-1.088625,0.170435,0.593641,0.170435,-0.341812,-1.088625,0.593641,0.170435,0.593641,0.170435,-0.341812,0.170435,0.000000,0.000000,0.170435,-0.341812,-1.088625


In [4]:
org_test_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.569715,-0.155969,1.586662,-0.110356,-0.021324,-0.412361,0.229132,-0.033252,-0.033252,-0.482741,0.699000,0.610469,0.630512,0.728991,0.268615,0.367391,0.733324,0.331370,0.885785,0.967358
1,-1.172108,-0.155969,-1.191184,-1.455578,0.023052,0.318783,0.229132,0.003738,0.169368,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,0.733324,0.331370,-0.692514,-0.506478
2,0.967358,-0.155969,-1.233862,-0.259956,-0.021324,-0.412361,-0.696401,0.003738,-0.141946,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-2.375949,-0.650735,-0.422533,0.569715
3,0.136166,-0.155969,1.586662,0.967358,-0.021324,0.318783,0.229132,0.003738,-0.141946,0.722046,0.699000,0.610469,-0.269859,0.728991,0.268615,0.367391,0.733324,-0.650735,0.885785,-0.390795
4,0.136166,-0.155969,-1.593819,1.363492,0.023052,0.318783,0.229132,0.003738,-0.141946,-0.482741,0.699000,0.610469,0.630512,-0.831894,0.268615,0.367391,-2.375949,-0.650735,-0.692514,0.136166


In [5]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1396 143 212 362
pod:  0.6306620209059234
pof:  0.09291747888239116
AUC:  0.768872271011766
recall:  0.6306620209059234
precision:  0.7168316831683168
F1-Score:  0.6709916589434661
accuracy:  0.8319924278277331


In [6]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1359 180 181 393
pod:  0.6846689895470384
pof:  0.11695906432748537
AUC:  0.7838549626097765
recall:  0.6846689895470384
precision:  0.6858638743455497
F1-Score:  0.6852659110723628
accuracy:  0.8291528632276385


In [7]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1409 130 274 300
pod:  0.5226480836236934
pof:  0.08447043534762833
AUC:  0.7190888241380327
recall:  0.5226480836236934
precision:  0.6976744186046512
F1-Score:  0.5976095617529881
accuracy:  0.8088026502602934


In [8]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1440 99 277 297
pod:  0.5174216027874564
pof:  0.06432748538011696
AUC:  0.7265470587036698
recall:  0.5174216027874564
precision:  0.75
F1-Score:  0.6123711340206185
accuracy:  0.8220539517274018


In [9]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1442 97 415 159
pod:  0.2770034843205575
pof:  0.06302794022092267
AUC:  0.6069877720498175
recall:  0.2770034843205575
precision:  0.62109375
F1-Score:  0.38313253012048193
accuracy:  0.7576904874585897


In [10]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1439 100 305 269
pod:  0.4686411149825784
pof:  0.0649772579597141
AUC:  0.7018319285114322
recall:  0.4686411149825784
precision:  0.7289972899728997
F1-Score:  0.5705196182396607
accuracy:  0.808329389493611


In [11]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0) 

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1461 78 344 230
pod:  0.40069686411149824
pof:  0.050682261208576995
AUC:  0.6750073014514606
recall:  0.40069686411149824
precision:  0.7467532467532467
F1-Score:  0.5215419501133787
accuracy:  0.8002839564600095


In [12]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix
 
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)    

Epoch 1/30
124/124 [==============================] - 2s 7ms/step - loss: 0.2621 - accuracy: 0.8935 - val_loss: 0.1815 - val_accuracy: 0.9249
Epoch 2/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1916 - accuracy: 0.9229 - val_loss: 0.1532 - val_accuracy: 0.9432
Epoch 3/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1693 - accuracy: 0.9353 - val_loss: 0.1425 - val_accuracy: 0.9503
Epoch 4/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1603 - accuracy: 0.9356 - val_loss: 0.1365 - val_accuracy: 0.9473
Epoch 5/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1513 - accuracy: 0.9404 - val_loss: 0.1362 - val_accuracy: 0.9483
Epoch 6/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1463 - accuracy: 0.9412 - val_loss: 0.1314 - val_accuracy: 0.9503
Epoch 7/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1426 - accuracy: 0.9419 - val_loss: 0.1340 - val_accuracy: 0.9503
Epoch 